In [1]:
# Einlesen der Bibliotheken
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from textblob import TextBlob


In [2]:
#pip install python-Levenshtein

In [3]:
#pip install textblob


In [55]:
df = pd.read_csv(r'C:\Users\alexa\OneDrive\Desktop\DEV\Power_BI_Dashboard\Data\raw\fedex_messy.csv', sep = ",")

In [3]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Source,Destination,Distance,Delivery_Status,fake_price,Item_Category,Delivery_Address,Customer_Name
0,2008,JANUAR,3,Thursday,2003.0,1955,2225,WN,335,150.0 min,8.0 minutes,IAD,TPA,810,Not Delivered,NONE,Books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis
1,2008,JANUAR,3,Thursday,754.0,735,1000,WN,3231,145.0 min.,19.0 Minutes,IAD,TPA,810 m,Delivered,556.34 €,Books,USNV Mosley\nFPO AA 74330,Michael Jackson
2,2008,JANUAR,3,Thu,NaN,620,750,WN,448,90.0 minutes,8.0 min,IND,BWI,515,not delivered,439.22 Euro,Clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason
3,2008,JANUAR,3,Thu,926.0,930,1100,WN,1746,90.0 Minutes,-4.0 min.,IND,BWI,515 Miles,NOT DELIVERED,297.53 Euro,Books,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill
4,2008,JAN,3,Thu,1829.0,1755,1925,WN,3920,90.0 minutes,34.0 min.,IND,BWI,515 m,Delivered,253.79,Office Supplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY


In [4]:
# alle Daten belassen, bei fehlenden Werten, alle Nan werte vereinheitlichen
# Zeit umformen in Stunden, Minuten und Sekunden.

In [56]:
fehlende_werte = df.isnull().sum()
print("Anzahl der fehlenden Werte in jeder Spalte:")
print(fehlende_werte)

Anzahl der fehlenden Werte in jeder Spalte:
Year                      42737
Month                     42611
DayofMonth                42400
DayOfWeek                 42977
Actual_Shipment_Time     122478
Planned_Shipment_Time     42741
Planned_Delivery_Time     42893
Carrier_Name              43053
Carrier_Num               42989
Planned_TimeofTravel      42879
Shipment_Delay            58611
Source                    42935
Destination               42744
Distance                  42823
Delivery_Status          122803
fake_price               324645
Item_Category             43012
Delivery_Address          42994
Customer_Name             42956
dtype: int64


In [57]:
df['Year'] = df["Year"].str.strip()
df["Year"].unique

<bound method Series.unique of 0          2008
1          2008
2          2008
3          2008
4          2008
           ... 
3604170    2008
3604171    2008
3604172    2008
3604173    2008
3604174    2008
Name: Year, Length: 3604175, dtype: object>

In [6]:
# Löscht Spalten, in denen mehr als 80% der Werte fehlen
df = df.dropna(axis=1, thresh=int(0.8 * len(df)))
# Löscht Zeilen, in denen mehr als 50% der Werte fehlen
df = df.dropna(axis=0, thresh=int(0.5 * len(df.columns)))

In [7]:
df["Month"].nunique

<bound method IndexOpsMixin.nunique of 0          JANUAR
1          JANUAR
2          JANUAR
3          JANUAR
4             JAN
            ...  
3604170       Jun
3604171      June
3604172       JUN
3604173      June
3604174       Jun
Name: Month, Length: 3604175, dtype: object>

In [8]:
df["Delivery_Status"].unique

<bound method Series.unique of 0          Not Delivered
1              Delivered
2          not delivered
3          NOT DELIVERED
4              Delivered
               ...      
3604170    not delivered
3604171    Not Delivered
3604172    not delivered
3604173    NOT DELIVERED
3604174                1
Name: Delivery_Status, Length: 3604175, dtype: object>

In [58]:
# Vereinheitlichung von Produktkategorien durch Kleinschreibung und Entfernung von Leerzeichen
df['Month'] = df["Month"].str.strip().str.lower()
df["Month"].unique

<bound method Series.unique of 0          januar
1          januar
2          januar
3          januar
4             jan
            ...  
3604170       jun
3604171      june
3604172       jun
3604173      june
3604174       jun
Name: Month, Length: 3604175, dtype: object>

In [59]:
# Umwandlung von Kategorienamen in konsistente Werte
category_mapping = {'jan': 'januar', 'jun': 'june',}
df['Month'] = df['Month'].replace(category_mapping)

In [60]:
category_mapping = {'januar': 'january', 'nan': 'NAN', '--' : 'NAN', "none": "NaN", "NaN": "nan", "feb": "february", "februar": "february", "mar": "march", "märz": "march",
                   "apr": "april", "mai": "may", "juni": "june"}
df['Month'] = df['Month'].replace(category_mapping)

In [61]:
df['Month'] = df['Month'].replace('nan', np.nan)

# Überprüfen, ob NaN-Werte in der Spalte vorkommen
nan_vorhanden = df['Month'].isna().any()

print("Gibt es NaN-Werte in der Spalte?:", nan_vorhanden)

Gibt es NaN-Werte in der Spalte?: True


In [62]:
df["Month"].str.title() # wieso funktioniert das nicht?

0          January
1          January
2          January
3          January
4          January
            ...   
3604170       June
3604171       June
3604172       June
3604173       June
3604174       June
Name: Month, Length: 3604175, dtype: object

In [48]:
df["Month"]

0          january
1          january
2          january
3          january
4          january
            ...   
3604170       june
3604171       june
3604172       june
3604173       june
3604174       june
Name: Month, Length: 3604175, dtype: object

In [63]:
df["DayofMonth"].unique()

array(['3', '--', nan, 'NONE', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '2', '1'],
      dtype=object)

In [64]:
df['DayofMonth'] = df['DayofMonth'].replace('nan', np.nan,)
df['DayofMonth'] = df['DayofMonth'].replace('--', np.nan,)
df['DayofMonth'] = df['DayofMonth'].replace('NONE', np.nan,)

In [65]:
df["DayofMonth"].unique()

array(['3', nan, '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '2', '1'], dtype=object)

In [66]:
df["DayOfWeek"].unique()

array(['Thursday', 'Thu', 'THURSDAY', '4', 'THU', nan, 'NONE', '--', '5',
       'Fri', 'FRIDAY', 'FRI', 'Friday', 'Sat', 'Saturday', '6', 'SAT',
       'SATURDAY', '7', 'SUNDAY', 'Sunday', 'Sun', 'SUN', 'Mon', 'MON',
       '1', 'MONDAY', 'Monday', 'Tuesday', 'TUE', 'Tue', '2', 'TUESDAY',
       'Wednesday', 'Wed', 'WED', 'WEDNESDAY', '3'], dtype=object)

In [67]:
category_mapping = {'Thu': 'Thursday', 'THURSDAY': 'Thursday', 'THU' : 'Thursday', 4: "Thursday", "Fri": "Friday", "FRI": "Friday", "FRIDAY": "Friday", 5: "Friday", "Sat": "Saturday", "SAT": "Saturday", "mar": "march", "märz": "march",
                   "SATURDAY": "Saturday", 6: "Saturday", "SUNDAY": "Sunday", "SUN": "Sunday", "Sun": "Sunday", 7: "Sunday", "MON": "Monday", "MONDAY": "Monday", "Mon": "Monday", 1: "Monday", "TUE": "Tuesday", "Tue": "Tuesday", "TUESDAY": "Tuesday", 2: "Tuesday", "Wed": "Wednesday", "WED": "Wednesday", "WEDNESDAY": "Wednesday", 3: "Wednesday", }
df['DayOfWeek'] = df['DayOfWeek'].replace(category_mapping)
wochentag_mapping = {
    '1': 'Monday',
    '2': 'Tuesday',
    '3': 'Wednesday',
    '4': 'Thursday',
    '5': 'Friday',
    '6': 'Saturday',
    '7': 'Sunday'
}
df['DayOfWeek'] = df['DayOfWeek'].replace(wochentag_mapping)


In [68]:
df["DayOfWeek"].unique()

array(['Thursday', nan, 'NONE', '--', 'Friday', 'Saturday', 'Sunday',
       'Monday', 'Tuesday', 'Wednesday'], dtype=object)

In [69]:
df['DayOfWeek'] = df['DayOfWeek'].replace('NONE', np.nan,)
df['DayOfWeek'] = df['DayOfWeek'].replace('--', np.nan,)

In [70]:
df["DayOfWeek"].unique() 

array(['Thursday', nan, 'Friday', 'Saturday', 'Sunday', 'Monday',
       'Tuesday', 'Wednesday'], dtype=object)

In [71]:
# Zunächst alle nicht-numerischen Werte und NaNs in echte NaN-Werte umwandeln
df['Actual_Shipment_Time'] = pd.to_numeric(df['Actual_Shipment_Time'], errors='coerce')
df["Actual_Shipment_Time"] = df["Actual_Shipment_Time"].astype(float)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3604175 entries, 0 to 3604174
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Year                   object 
 1   Month                  object 
 2   DayofMonth             object 
 3   DayOfWeek              object 
 4   Actual_Shipment_Time   float64
 5   Planned_Shipment_Time  object 
 6   Planned_Delivery_Time  object 
 7   Carrier_Name           object 
 8   Carrier_Num            object 
 9   Planned_TimeofTravel   object 
 10  Shipment_Delay         object 
 11  Source                 object 
 12  Destination            object 
 13  Distance               object 
 14  Delivery_Status        object 
 15  fake_price             object 
 16  Item_Category          object 
 17  Delivery_Address       object 
 18  Customer_Name          object 
dtypes: float64(1), object(18)
memory usage: 522.5+ MB


In [73]:
df["Planned_Shipment_Time"] = pd.to_numeric(df['Planned_Shipment_Time'], errors='coerce')
df["Planned_Shipment_Time"] = df["Planned_Shipment_Time"].astype(float)

In [74]:
df["Planned_Delivery_Time"] = pd.to_numeric(df['Planned_Delivery_Time'], errors='coerce')
df["Planned_Delivery_Time"] = df["Planned_Delivery_Time"].astype(float)

In [75]:
df["Carrier_Num"] = pd.to_numeric(df['Carrier_Num'], errors='coerce')
# Option 1: Fill NaNs with a default value, e.g., 0, then convert to integer
df['Carrier_Num'] = df['Carrier_Num'].fillna(0).astype(int)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3604175 entries, 0 to 3604174
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Year                   object 
 1   Month                  object 
 2   DayofMonth             object 
 3   DayOfWeek              object 
 4   Actual_Shipment_Time   float64
 5   Planned_Shipment_Time  float64
 6   Planned_Delivery_Time  float64
 7   Carrier_Name           object 
 8   Carrier_Num            int32  
 9   Planned_TimeofTravel   object 
 10  Shipment_Delay         object 
 11  Source                 object 
 12  Destination            object 
 13  Distance               object 
 14  Delivery_Status        object 
 15  fake_price             object 
 16  Item_Category          object 
 17  Delivery_Address       object 
 18  Customer_Name          object 
dtypes: float64(3), int32(1), object(15)
memory usage: 508.7+ MB


ValueError: could not convert string to float: '145.0.'

In [77]:
# Bereinigung und Standardisierung der Einheiten auf "min"
df['Planned_TimeofTravel'] = df['Planned_TimeofTravel'].str.extract(r'(\d+\.?\d*)')  # extrahiert die Zahl als String
df['Planned_TimeofTravel'] = df['Planned_TimeofTravel'].astype(float)  # konvertiert in float, um sicherzustellen, dass alle Werte numerisch sind



In [78]:
# Bereinigung und Standardisierung der Einheiten auf "min"
df['Shipment_Delay'] = df['Shipment_Delay'].str.extract(r'(\d+\.?\d*)')  # extrahiert die Zahl als String
df['Shipment_Delay'] = df['Shipment_Delay'].astype(float)  # konvertiert in float, um sicherzustellen, dass alle Werte numerisch sind



In [79]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Source,Destination,Distance,Delivery_Status,fake_price,Item_Category,Delivery_Address,Customer_Name
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,8.0,IAD,TPA,810,Not Delivered,NONE,Books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,19.0,IAD,TPA,810 m,Delivered,556.34 €,Books,USNV Mosley\nFPO AA 74330,Michael Jackson
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,8.0,IND,BWI,515,not delivered,439.22 Euro,Clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,4.0,IND,BWI,515 Miles,NOT DELIVERED,297.53 Euro,Books,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,34.0,IND,BWI,515 m,Delivered,253.79,Office Supplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY


In [80]:
# Bereinigung und Standardisierung der Einheiten auf "min"
df['Distance'] = df['Distance'].str.extract(r'(\d+\.?\d*)')  # extrahiert die Zahl als String
df['Distance'] = df['Distance'].astype(float)  # konvertiert in float, um sicherzustellen, dass alle Werte numerisch sind

In [81]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Source,Destination,Distance,Delivery_Status,fake_price,Item_Category,Delivery_Address,Customer_Name
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,8.0,IAD,TPA,810.0,Not Delivered,NONE,Books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,19.0,IAD,TPA,810.0,Delivered,556.34 €,Books,USNV Mosley\nFPO AA 74330,Michael Jackson
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,8.0,IND,BWI,515.0,not delivered,439.22 Euro,Clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,4.0,IND,BWI,515.0,NOT DELIVERED,297.53 Euro,Books,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,34.0,IND,BWI,515.0,Delivered,253.79,Office Supplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3604175 entries, 0 to 3604174
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Year                   object 
 1   Month                  object 
 2   DayofMonth             object 
 3   DayOfWeek              object 
 4   Actual_Shipment_Time   float64
 5   Planned_Shipment_Time  float64
 6   Planned_Delivery_Time  float64
 7   Carrier_Name           object 
 8   Carrier_Num            int32  
 9   Planned_TimeofTravel   float64
 10  Shipment_Delay         float64
 11  Source                 object 
 12  Destination            object 
 13  Distance               float64
 14  Delivery_Status        object 
 15  fake_price             object 
 16  Item_Category          object 
 17  Delivery_Address       object 
 18  Customer_Name          object 
dtypes: float64(6), int32(1), object(12)
memory usage: 508.7+ MB


In [82]:
df["fake_price"].unique()

array(['NONE', '556.34 €', '439.22 Euro', ..., '761.93 EURO',
       '254.46 Euro', '307.40 DOLLAR'], dtype=object)

In [83]:
# Funktion, um die Währung zu extrahieren
def extract_currency(value):
    if isinstance(value, str):  # Nur Strings verarbeiten
        match = re.search(r'(€|Euro|DOLLAR|EURO)', value, re.IGNORECASE)
        return match.group(0).upper() if match else None
    return None  # Für andere Datentypen None zurückgeben

# Funktion, um den Preis zu bereinigen
def clean_price(value):
    if isinstance(value, str):  # Nur Strings verarbeiten
        # Entfernt alles außer Zahlen, Punkt und Komma
        cleaned = re.sub(r'[^\d.,]', '', value)
        # Ersetzt Komma durch Punkt (falls nötig)
        cleaned = cleaned.replace(',', '.')
        return float(cleaned) if cleaned else None
    elif isinstance(value, (int, float)):  # Zahlen direkt zurückgeben
        return float(value)
    return None  # Für andere Datentypen None zurückgeben

# Neue Spalten hinzufügen
df['currency'] = df['fake_price'].apply(extract_currency)
df['price'] = df['fake_price'].apply(clean_price)

In [84]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,...,Source,Destination,Distance,Delivery_Status,fake_price,Item_Category,Delivery_Address,Customer_Name,currency,price
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,...,IAD,TPA,810.0,Not Delivered,NONE,Books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis,None,NaN
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,...,IAD,TPA,810.0,Delivered,556.34 €,Books,USNV Mosley\nFPO AA 74330,Michael Jackson,€,556.34
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,...,IND,BWI,515.0,not delivered,439.22 Euro,Clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason,EURO,439.22
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,...,IND,BWI,515.0,NOT DELIVERED,297.53 Euro,Books,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill,EURO,297.53
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,...,IND,BWI,515.0,Delivered,253.79,Office Supplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY,None,253.79


In [85]:
# lösche die Spalte fake_price
df = df.drop(columns=['fake_price'])

In [86]:
# Ersetzt None durch NaN
df['currency'] = df['currency'].replace([None], np.nan)
# Standardisiert alle nicht-None-Werte zu "€"
df['currency'] = df['currency'].replace(['EURO', 'DOLLAR'], '€')

In [87]:
df["currency"].unique()

array([nan, '€'], dtype=object)

In [ ]:
# wie kann ich herausfinden, ob verschiedene Währungen vorliegen?

In [88]:
df["Delivery_Status"].unique()

array(['Not Delivered', 'Delivered', 'not delivered', 'NOT DELIVERED',
       '1', 'not Delivered', '0', 'DELIVERED', 'deliverd', 'NONE', nan,
       '--'], dtype=object)

In [89]:
df['Delivery_Status'] = df["Delivery_Status"].str.lower()

In [90]:
category_mapping = {'deliverd': 'delivered'}
delivery_mapping = {
    '1': 'delivered',
    '0': 'not delivered'}
df['DayOfWeek'] = df['DayOfWeek'].replace(delivery_mapping)
df['Delivery_Status'] = df['Delivery_Status'].replace(category_mapping)
df['Delivery_Status'] = df['Delivery_Status'].replace('none', np.nan,)
df['Delivery_Status'] = df['Delivery_Status'].replace('--', np.nan,)

In [91]:
df["Item_Category"].nunique()
#ich habe eine Spalte namens item categories. In dieser Spalte habe ich 3980 einzigartige Einträge.
#Doch einige Einträge sind falsch geschrieben, wie zum Beispiel anstelle von Clothing, steht Clothink. Mit der unique Methode 
# kann ich natürlich nicht alle abbilden. Wie kann ich diese einzigartigen Einträge in ähnliche Gruppen gruppieren?

3918

In [92]:
# NaN-Werte behandeln: Ersetzen durch einen leeren String oder entfernen
df['Item_Category'].fillna('', inplace=True)  # Ersetzt NaN mit einem leeren String
# TF-IDF-Vektorisierung
vectorizer = TfidfVectorizer(max_features=1000)  # Nur die 1000 häufigsten Features
X = vectorizer.fit_transform(df['Item_Category'])


C:\Users\alexa\AppData\Local\Temp\ipykernel_8120\2313795604.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Item_Category'].fillna('', inplace=True)  # Ersetzt NaN mit einem leeren String


In [47]:
# Clustering mit DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=1, metric="cosine")
labels = dbscan.fit_predict(X)

# Ergebnisse anzeigen
df['cluster'] = labels
print(df.sort_values(by='cluster'))


KeyboardInterrupt



In [48]:
# Überprüfen auf verbleibende NaN-Werte
print(df['Item_Category'].isnull().sum())

0


In [49]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100)  # Wähle eine geeignete Anzahl von Komponenten
X_reduced = svd.fit_transform(X)

# Clustering mit DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=1, metric="cosine")
labels = dbscan.fit_predict(X_reduced)

# Ergebnisse anzeigen
df['cluster'] = labels

KeyboardInterrupt: 

In [45]:
# Leerstellen entfernen und kleinschreibung
df['Item_Category'] = df["Item_Category"].str.strip().str.lower()
# 12. Rechtschreibkorrekturen in Textdaten, lädt Ewigkeiten
#df['Item_Category'] = df['Item_Category'].apply(lambda x: str(TextBlob(x).correct()))
# Entferne Leerzeichen vor, nach und zwischen den Wörtern in der Spalte
df['Item_Category'] = df['Item_Category'].str.replace(r'\s+', '', regex=True)

In [46]:
# Liste der einzigartigen Einträge
unique_items = df['Item_Category'].unique()

# Leeres Dictionary, um Gruppen zu speichern
groups = {}

# Loop durch jeden einzigartigen Eintrag und finde ähnliche Einträge
for item in unique_items:
    # Suche nach ähnlichen Einträgen, die zu diesem Item passen (mit Schwellenwert von 80)
    similar_items = process.extract(item, unique_items, limit=10, scorer=fuzz.ratio)
    similar_items = [i[0] for i in similar_items if i[1] >= 80]  # Nur mit Ähnlichkeit >= 80 aufnehmen
    
    # Verwende das erste Element der ähnlichen Einträge als Gruppenschlüssel
    key = similar_items[0]
    if key in groups:
        groups[key].update(similar_items)
    else:
        groups[key] = set(similar_items)

# Konvertiere das Dictionary in eine Liste von Einträgen
for key, items in groups.items():
    print(f"Gruppe: {key} -> {items}")

Gruppe: books -> {'bdoks', 'bosks', 'booxs', 'bxoks', 'bzoks', 'bookj', 'books', 'boois', 'bvoks', 'pooks'}
Gruppe: clothing -> {'cllthing', 'clothizg', 'clothisg', 'clotting', 'clothing', 'coothing', 'clotcing', 'clrthing', 'cloohing', 'jlothing'}
Gruppe: officesupplies -> {'officesupplies', 'officehsupplies', 'officeasupplies', 'officensupplies', 'officeisupplies', 'officeysupplies', 'officexsupplies', 'officetsupplies', 'officezsupplies', 'officeksupplies'}
Gruppe: furniture -> {'furnigure', 'funniture', 'furnixure', 'fucniture', 'furniturp', 'furnituge', 'furnmture', 'furtiture', 'furnituke', 'furniture'}
Gruppe: toys -> {'toys'}
Gruppe: automotive -> {'autdmotive', 'automotivo', 'autbmotive', 'automotive', 'autxmotive', 'automotivn', 'autoeotive', 'automocive', 'automotivr', 'autolotive'}
Gruppe: electronics -> {'elecuronics', 'edectronics', 'ylectronics', 'electrocics', 'electronics', 'electrolics', 'elecfronics', 'elyctronics', 'electnonics', 'electkonics'}
Gruppe: health -> {'h

In [47]:
# Erstelle ein Mapping für die Standardisierung
mapping = {}
for key, items in groups.items():
    for item in items:
        mapping[item] = key  # Weise alle ähnlichen Items dem Gruppenschlüssel zu

# Standardisiere die Werte im DataFrame
df['Standardized_Item_Category'] = df['Item_Category'].map(mapping)

# Zeige das aktualisierte DataFrame an
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Source,Destination,Distance,Delivery_Status,fake_price,Item_Category,Delivery_Address,Customer_Name,Standardized_Item_Category
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,8.0,IAD,TPA,810.0,not delivered,NaN,books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis,rooks
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,19.0,IAD,TPA,810.0,delivered,556.34,books,USNV Mosley\nFPO AA 74330,Michael Jackson,rooks
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,8.0,IND,BWI,515.0,not delivered,439.22,clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason,clothrng
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,4.0,IND,BWI,515.0,not delivered,297.53,books,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill,rooks
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,34.0,IND,BWI,515.0,delivered,253.79,officesupplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY,ofcicesupplies


In [48]:
df["Standardized_Item_Category"].nunique()
# reduziert von 3900 auf 2239 ohne strip() und lower() auf Item_Category
# reduziert auf 1637 mit lower, strip und replace method

1637

In [60]:
# erneute Anwendung

# Liste der einzigartigen Einträge, NaN-Werte entfernen
unique_items = df['Standardized_Item_Category'].dropna().unique()

# Leeres Dictionary, um Gruppen zu speichern
groups = {}

# Loop durch jeden einzigartigen Eintrag und finde ähnliche Einträge
for item in unique_items:
    # Suche nach ähnlichen Einträgen, die zu diesem Item passen (mit Schwellenwert von 80)
    similar_items = process.extract(item, unique_items, limit=10, scorer=fuzz.ratio)
    similar_items = [i[0] for i in similar_items if i[1] >= 80]  # Nur mit Ähnlichkeit >= 80 aufnehmen
    
    # Verwende das erste Element der ähnlichen Einträge als Gruppenschlüssel
    key = similar_items[0]
    if key in groups:
        groups[key].update(similar_items)
    else:
        groups[key] = set(similar_items)

# Konvertiere das Dictionary in eine Liste von Einträgen
for key, items in groups.items():
    print(f"Gruppe: {key} -> {items}")

Gruppe: rooks -> {'gooks', 'rooks', 'hooks', 'uooks', 'eooks', 'yooks', 'fooks', 'mooks', 'jooks', 'aooks'}
Gruppe: clothrng -> {'clothxng', 'clothkng', 'clothmng', 'clothong', 'clothwng', 'clothrng', 'clothung', 'clothhng', 'clothfng', 'clothpng'}
Gruppe: ofcicesupplies -> {'ofcicesupplies', 'ofuicesupplies', 'ofiicesupplies', 'ofqicesupplies', 'ofbicesupplies', 'ofnicesupplies', 'ofzicesupplies', 'ofkicesupplies', 'ofwicesupplies', 'ofhicesupplies'}
Gruppe: fuwniture -> {'fuyniture', 'fuuniture', 'fugniture', 'fusniture', 'fuhniture', 'fwrniture', 'fulniture', 'fubniture', 'fuwniture', 'furwiture'}
Gruppe: toys -> {'toys'}
Gruppe: autohotive -> {'authmotive', 'autozotive', 'automhtive', 'autouotive', 'autocotive', 'autosotive', 'autohotive', 'autovotive', 'autogotive', 'autonotive'}
Gruppe: elejtronics -> {'eleztronics', 'elecjronics', 'eleetronics', 'elebtronics', 'eleqtronics', 'elejtronics', 'elentronics', 'eleotronics', 'eleytronics', 'elemtronics'}
Gruppe: heagth -> {'heanth', '

In [61]:
# Erstelle ein Mapping für die Standardisierung
mapping = {}
for key, items in groups.items():
    for item in items:
        mapping[item] = key  # Weise alle ähnlichen Items dem Gruppenschlüssel zu

# Standardisiere die Werte im DataFrame
df['second_standardized_Item_Category'] = df['Standardized_Item_Category'].map(mapping)

# Zeige das aktualisierte DataFrame an
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,...,Source,Destination,Distance,Delivery_Status,fake_price,Item_Category,Delivery_Address,Customer_Name,Standardized_Item_Category,second_standardized_Item_Category
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,...,IAD,TPA,810.0,not delivered,NaN,books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis,rooks,wooks
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,...,IAD,TPA,810.0,delivered,556.34,books,USNV Mosley\nFPO AA 74330,Michael Jackson,rooks,wooks
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,...,IND,BWI,515.0,not delivered,439.22,clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason,clothrng,clothirg
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,...,IND,BWI,515.0,not delivered,297.53,books,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill,rooks,wooks
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,...,IND,BWI,515.0,delivered,253.79,officesupplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY,ofcicesupplies,ofmicesupplies


In [62]:
df["second_standardized_Item_Category"].nunique()
#reduziert von 2200 auf 1161 ohne strip() und lower() auf Item_Category
# reduziert auf 767 mit stip , lower und replace auf item category

767

In [53]:
''

 "Standardwörter" automatisch zu erkennen, kannst du eine Clustering-Methode anwenden, um ähnliche Begriffe zu gruppieren und dann für jede Gruppe automatisch ein zentrales oder repräsentatives Wort als Standard zu wählen. Ein gängiger Ansatz für solch eine automatisierte Gruppierung basiert auf einem Ähnlichkeits-Scoring und Clustering.

Hier sind die Schritte und ein Beispielcode für ein solches Verfahren:

Schritte zum automatischen Clustering und Standardisierung
Ähnlichkeitsmatrix erstellen: Berechne für alle Einträge die Ähnlichkeiten zueinander.
Clustering anwenden: Verwende ein Clustering-Verfahren (z.B. Agglomeratives Clustering) basierend auf den Ähnlichkeiten.

Standardwörter festlegen: Wähle für jede Gruppe ein repräsentatives Wort aus, z. B. das häufigste Wort oder den Eintrag, der am nächsten zum Zentrum der Gruppe liegt.
''


''

In [63]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# Beispiel-DataFrame mit der Spalte 'Standardized_Item_Category'

# 1. Einträge vorbereiten und Duplikate entfernen
unique_items = df['second_standardized_Item_Category'].dropna().unique()

# 2. TF-IDF Vektorisierung, um Text in Zahlenform zu bringen
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
tfidf_matrix = vectorizer.fit_transform(unique_items)

# 3. Berechne die Ähnlichkeitsmatrix (Kosinus-Ähnlichkeit)
similarity_matrix = cosine_similarity(tfidf_matrix)

# 4. Clustering mit Agglomerative Clustering
# Wähle die Anzahl der Cluster oder den Abstandsschwellenwert
clustering = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='complete', distance_threshold=0.4)
labels = clustering.fit_predict(1 - similarity_matrix)

# 5. Gruppieren und Standardwörter festlegen
groups = {}
for label, item in zip(labels, unique_items):
    if label in groups:
        groups[label].append(item)
    else:
        groups[label] = [item]

# Wähle ein Standardwort pro Gruppe, z. B. das häufigste
standardized_mapping = {}
for items in groups.values():
    # Du kannst hier eine andere Logik verwenden, um das repräsentative Wort zu wählen
    standard_word = min(items, key=len)  # z.B. das kürzeste Wort als Standard wählen
    for item in items:
        standardized_mapping[item] = standard_word

# 6. Original-Spalte mit den Standardwerten aktualisieren
df['second_standardized_Item_Category'] = df['second_standardized_Item_Category'].map(standardized_mapping).fillna(df['second_standardized_Item_Category'])

# Ausgabe der DataFrame nach der Standardisierung
print(df)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


         Year    Month DayofMonth DayOfWeek  Actual_Shipment_Time  \
0        2008  january          3  Thursday                2003.0   
1        2008  january          3  Thursday                 754.0   
2        2008  january          3  Thursday                   NaN   
3        2008  january          3  Thursday                 926.0   
4        2008  january          3  Thursday                1829.0   
...       ...      ...        ...       ...                   ...   
3604170  2008     june         19  Thursday                1059.0   
3604171  2008     june         19  Thursday                 555.0   
3604172  2008     june         19  Thursday                 821.0   
3604173  2008     june         19  Thursday                 718.0   
3604174  2008     june         19  Thursday                1127.0   

         Planned_Shipment_Time  Planned_Delivery_Time Carrier_Name  \
0                       1955.0                 2225.0           WN   
1                        735.0 

In [64]:
df["second_standardized_Item_Category"].nunique()

639

In [65]:
df["second_standardized_Item_Category"].unique()

array(['wooks', 'clothirg', 'ofmicesupplies', 'fuqniture', 'toys',
       'autototive', 'eleptronics', 'heaeth', 'gooceries', 'applianoes',
       'none', 'tsys', 'spxrts', 'bolks', '', 'jewexry', 'ylothing',
       'electrouics', 'tkys', 'cloteing', 'clothitg', 'furnituie', 'tods',
       'bookp', 'clozhing', 'booos', 'appjiances', 'joys', 'clfthing',
       'offiresupplies', 'appliancbs', 'clothinb', 'officegsupplies',
       'tbys', 'furnifure', 'electvonics', 'lports', 'jevelry', 'healph',
       'grocwries', 'elecyronics', 'furniturw', 'furyiture', 'goys',
       'healtt', 'sporte', 'aueomotive', 'bobks', 'nroceries', 'hwalth',
       'toks', 'automotivz', 'heqlth', 'toxs', 'officesvpplies',
       'aphliances', 'grocerkes', 'thys', 'spouts', 'boaks', 'cnothing',
       'clothihg', 'clothgng', 'applaances', 'boys', 'shorts',
       'enectronics', 'toyj', 'tyys', 'groceiies', 'sporis', 'bowks',
       'bookg', 'xoys', 'bqoks', 'tpys', 'elpctronics', 'rurniture',
       'officlsuppl

In [66]:

from difflib import get_close_matches

# Assuming 'df' is your DataFrame
# The column name is "second_standardized_Item_Category"

# List of correctly spelled categories for reference
correct_categories = [
    'books', 'clothing', 'office supplies', 'furniture', 'toys', 
    'automotive', 'electronics', 'health', 'groceries', 'appliances', 'jewelry', 'sports'
]

# Function to correct each item in the column
def correct_spelling(category):
    # Get the closest match from the correct categories, with a similarity threshold of 0.6
    matches = get_close_matches(category, correct_categories, n=1, cutoff=0.6)
    return matches[0] if matches else category

# Apply the function to the "second_standardized_Item_Category" column
df['second_standardized_Item_Category'] = df['second_standardized_Item_Category'].apply(correct_spelling)

# Display the corrected column
print(df['second_standardized_Item_Category'])


0                    books
1                    books
2                 clothing
3                    books
4          office supplies
                ...       
3604170            jewelry
3604171               toys
3604172             sports
3604173           clothing
3604174           clothing
Name: second_standardized_Item_Category, Length: 3604175, dtype: object


In [69]:
df['second_standardized_Item_Category'].unique()

array(['books', 'clothing', 'office supplies', 'furniture', 'toys',
       'automotive', 'electronics', 'health', 'groceries', 'appliances',
       'none', 'sports', '', 'jewelry'], dtype=object)

In [70]:
# Drop the old 'Item_Category' column
df = df.drop(columns=['Item_Category'])

# Rename 'second_standardized_Item_Category' to 'Item_Category'
df = df.rename(columns={'second_standardized_Item_Category': 'Item_Category'})

# Display the updated DataFrame
print(df.head())

   Year    Month DayofMonth DayOfWeek  Actual_Shipment_Time  \
0  2008  january          3  Thursday                2003.0   
1  2008  january          3  Thursday                 754.0   
2  2008  january          3  Thursday                   NaN   
3  2008  january          3  Thursday                 926.0   
4  2008  january          3  Thursday                1829.0   

   Planned_Shipment_Time  Planned_Delivery_Time Carrier_Name  Carrier_Num  \
0                 1955.0                 2225.0           WN          335   
1                  735.0                 1000.0           WN         3231   
2                  620.0                  750.0           WN          448   
3                  930.0                 1100.0           WN         1746   
4                 1755.0                 1925.0           WN         3920   

   Planned_TimeofTravel  Shipment_Delay Source Destination  Distance  \
0                 150.0             8.0    IAD         TPA     810.0   
1             

In [71]:
df = df.drop(columns=['Standardized_Item_Category'])
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Source,Destination,Distance,Delivery_Status,fake_price,Delivery_Address,Customer_Name,Item_Category
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,8.0,IAD,TPA,810.0,not delivered,NaN,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",Gary Davis,books
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,19.0,IAD,TPA,810.0,delivered,556.34,USNV Mosley\nFPO AA 74330,Michael Jackson,books
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,8.0,IND,BWI,515.0,not delivered,439.22,"7340 Amcer Plaza\nWest Katherine, VI 83367",Amy Mason,clothing
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,4.0,IND,BWI,515.0,not delivered,297.53,"PSC 5740, Box 7993\nAPO AA 8a834",Bobby Hill,books
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,34.0,IND,BWI,515.0,delivered,253.79,4282 William Village Apt. 446\nNew Sharonchest...,KATIE KIRBY,office supplies


In [99]:
df["Customer_Name"].str.lower().str.strip()
df['Customer_Name'] = df['Customer_Name'].str.replace(r'\s+', '', regex=True)

In [100]:
df['Customer_Name'].nunique()

1186588

In [101]:
df['Customer_Name'].unique()

array(['GaryDavis', 'MichaelJackson', 'AmyMason', ..., 'JesnaHess',
       'CherylFreemanDDS', 'EthanWeaver'], dtype=object)

In [102]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,Actual_Shipment_Time,Planned_Shipment_Time,Planned_Delivery_Time,Carrier_Name,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Source,Destination,Distance,Delivery_Status,Item_Category,Delivery_Address,Customer_Name,currency,price
0,2008,january,3,Thursday,2003.0,1955.0,2225.0,WN,335,150.0,8.0,IAD,TPA,810.0,not delivered,Books,"55689 slan Plaza Suite 062\nEast Curtisburgh, ...",GaryDavis,NaN,NaN
1,2008,january,3,Thursday,754.0,735.0,1000.0,WN,3231,145.0,19.0,IAD,TPA,810.0,delivered,Books,USNV Mosley\nFPO AA 74330,MichaelJackson,€,556.34
2,2008,january,3,Thursday,NaN,620.0,750.0,WN,448,90.0,8.0,IND,BWI,515.0,not delivered,Clothing,"7340 Amcer Plaza\nWest Katherine, VI 83367",AmyMason,€,439.22
3,2008,january,3,Thursday,926.0,930.0,1100.0,WN,1746,90.0,4.0,IND,BWI,515.0,not delivered,Books,"PSC 5740, Box 7993\nAPO AA 8a834",BobbyHill,€,297.53
4,2008,january,3,Thursday,1829.0,1755.0,1925.0,WN,3920,90.0,34.0,IND,BWI,515.0,delivered,Office Supplies,4282 William Village Apt. 446\nNew Sharonchest...,KATIEKIRBY,NaN,253.79


In [81]:
# Liste der einzigartigen Einträge
unique_items = df['Customer_Name'].dropna().unique()

# Leeres Dictionary, um Gruppen zu speichern
groups = {}

# Loop durch jeden einzigartigen Eintrag und finde ähnliche Einträge
for item in unique_items:
    # Suche nach ähnlichen Einträgen, die zu diesem Item passen (mit Schwellenwert von 80)
    similar_items = process.extract(item, unique_items, limit=10, scorer=fuzz.ratio)
    similar_items = [i[0] for i in similar_items if i[1] >= 80]  # Nur mit Ähnlichkeit >= 80 aufnehmen
    
    # Verwende das erste Element der ähnlichen Einträge als Gruppenschlüssel
    key = similar_items[0]
    if key in groups:
        groups[key].update(similar_items)
    else:
        groups[key] = set(similar_items)

# Konvertiere das Dictionary in eine Liste von Einträgen
for key, items in groups.items():
    print(f"Gruppe: {key} -> {items}")



# Auch nach 30 Minuten kein Ergebnis...

KeyboardInterrupt: 

In [ ]:
# Erstelle ein Mapping für die Standardisierung
mapping = {}
for key, items in groups.items():
    for item in items:
        mapping[item] = key  # Weise alle ähnlichen Items dem Gruppenschlüssel zu

# Standardisiere die Werte im DataFrame
df['second_standardized_Item_Category'] = df['Customer_Name'].map(mapping)

# Zeige das aktualisierte DataFrame an
df.head()